<a href="https://colab.research.google.com/github/khusanovich/modul_3.1/blob/main/3_2_1_dropna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 3-MODUL. MA’LUMOTLARGA ISHLOV BERISH

### 3.2 Ma'lumotlarni tozalash va tayyorlash

Data Science muhandisi sifatida sizning 80% vaqtingiz ma'lumotlarga ishlov berish bilan o'tadi. Jumladan, ma'lumotlarni o'qish, kamchiliklarni bartaraf etish, ma'lumotlarni shakl o'zgartirish, matnlar bilan ishlash va hokazo. 

Navbatdagi bo'limda aynan shu mavzularga batafsil to'xtalamiz.

## Mavjud bo'lmagan qiymatlar bilan ishlash

Ko'p holatlarda jamlangan ma'lumotlar to'liq bo'lmaydi. Minglab qatorlar orasida albatta qandaydur qiymatlar tushib qolishi tabiiy (odam xatosi, ma'lumot mavjud emasligi va hokazo). `pandas` bunday qiymatlar bilan ishlashni bir muncha osonlashtirsada, yakuniy datasetda bu akmchiliklarni albatta bartaraf etish kerak. 

`pandas` DataFramelar mavjud bo'lmagan sonli qiymatlar o'rniga `NaN` (not available - mvajud emas) degna qiymatni qo'yib ketadi va statistik ma'lumotlarni chiqarishda bunday qiymatlarni inobatga olmaydi.

In [1]:
import pandas as pd

Yangi DF yaratamiz. Jadval tarkibidagi `NaN` qiymatlarga ahamiyat bering.

In [2]:
df = pd.read_csv("https://github.com/anvarnarz/praktikum_datasets/raw/main/automobile_data_nan.csv", index_col=0)
df.head()

,company,body-style,wheel-base,length,engine-type,num-of-cylinders,horsepower,average-mileage,price
index,,,,,,,,,
0,alfa-romero,convertible,88.6,168.8,dohc,four,NaN,21.0,13495.0
1,alfa-romero,convertible,NaN,168.8,dohc,four,111.0,21.0,16500.0
2,alfa-romero,hatchback,94.5,171.2,ohcv,six,154.0,NaN,16500.0
3,audi,sedan,99.8,176.6,ohc,four,102.0,24.0,13950.0
4,audi,sedan,99.4,176.6,ohc,five,NaN,NaN,17450.0


**Mavjud bo'lmagan qiymatlarga ishlov berishning yagona, universal formulasi yo'q.**

Bunday holatda sizning birinchi savolingiz "Nima uchun mu qiymat mavjud emas? Ma'lumot yozib olinmaganmi yoki umuman mavjud emasmi?" bo'lishi kerak. Misol uchun, siz so'rovnoma yordamida odamlar haqida ma'lumotlar jamlayapsiz, so'rovnomangizda "nikohdan o'tgan yilingiz" degan bo'lim bor. Tabiiyki, turmush qurmaghanlar uchun bu qiymat mabjud bo'lmaydi va bu xato emas. Aksincha, "tug'ilgan yilingiz" qiymati mavjud bo'lmasligi esa, ma'lumot mavjud lekin yozib olinmaganini anglatadi.

Data Science muhandisi sifatida siz mavjud bo'lmagan ma'lumotlarni topishga, buning imkoni bo'lmasa ularni approksimasiya qilishga (tahminiy qiymat topishga) harakat qilishingiz, buni ham iloji bo'lmasa bunday ustun (yoki qatorlardan) voz kechishni hal qilishingiz kerak bo'ladi. 

Avvalo, keling, DF da mavjud bo'lmagan (`NaN`) qiymatlar sonini topaylik.

#### `.isnul()`
`.isnull()` metodi yordamida ustun qiymati `NaN` ga teng (`True`) yoki yo'q (`False`) ekanini tekshiramiz. Ya'ni, `NaN` qiymat o'rnida `True` ko'ramiz. 

In [3]:
df.isnull()

,company,body-style,wheel-base,length,engine-type,num-of-cylinders,horsepower,average-mileage,price
index,,,,,,,,,
0,False,False,False,False,False,False,True,False,False
1,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...
81,False,False,False,False,False,False,False,False,False
82,False,False,True,False,True,False,False,False,False
86,False,False,False,True,False,False,False,True,False


Boolean qiymatlar `True=1`, `False=0` bo'lgani uchun `.sum()` metodi yordamida har ustundagi jami `NaN` qiymatlar sonini ko'rishimiz mumkin.

In [4]:
df.isnull().sum()

company              0
body-style           0
wheel-base          15
length              13
engine-type         12
num-of-cylinders     0
horsepower          16
average-mileage     15
price                3
dtype: int64

Ko'rib turganingizdek, `horsepower` ustunida 16 ta, `wheel-base` va `average-milega` ustunlarida 15 tadan va hokazo qiymatlar mavjud emas. 

Keling bu ko'rsatkichlarni jami ma'lumotlarning foizi sifatida chiqaramiz.

In [7]:
(df.isnull().sum()/df.shape[0])*100

company              0.000000
body-style           0.000000
wheel-base          24.590164
length              21.311475
engine-type         19.672131
num-of-cylinders     0.000000
horsepower          26.229508
average-mileage     24.590164
price                4.918033
dtype: float64

Yuoqirdan ba'zi ustunlardagi `NaN` qiymatlar soni 25% dan ham ko'p ekanini ko'rishimiz mumkin. Bu, albatta, yaxshi emas.

`pandas` da `NaN` qiymatlar bilan ishlash uchun bir nechta metodlar bor:
- `dropna` - `NaN` qiymatlar bor ustun yoki qatorlarni o'chirib tashlash 
- `fillna` - `NaN` qiymmatlarni boshqa qiymatlar yordamida to'ldirib ketish (misol uchun avvalgi yoki keyingi qator qiymatlari)
- `isnull` - `NaN` qiymatlarni filtrlash (`NaN` qiymat `True` qaytaradi)
- `notnull` - `isnull` ning aksi

### `.dropna()` `NaN` qiymatlarni tashlab yuborish
Ushbu metod yordamida `NaN` qiymatlar bor ustun yoki qatorlarni butunlay tashlab yuborish mumkin.

Qatorlardan voz kechish uchun `.dropna()` funksiyasini parametrsiz chaqirish kifoya:

In [9]:
df2 = df.dropna()
df2.shape

(14, 9)

Ko'rib turganingizdek, `.dropna()` funksiyasini qo'llaganimizdan so'ng asl DF dan 14 ta qator qoldi xolos. Kelin ularni chiqaramiz:

In [10]:
df2

,company,body-style,wheel-base,length,engine-type,num-of-cylinders,horsepower,average-mileage,price
index,,,,,,,,,
3,audi,sedan,99.8,176.6,ohc,four,102.0,24.0,13950.0
5,audi,sedan,99.8,177.3,ohc,five,110.0,19.0,15250.0
6,audi,wagon,105.8,192.7,ohc,five,110.0,19.0,18920.0
17,chevrolet,hatchback,94.5,155.9,ohc,four,70.0,38.0,6295.0
19,dodge,hatchback,93.7,157.3,ohc,four,68.0,31.0,6377.0
28,honda,sedan,96.5,175.4,ohc,four,101.0,24.0,12945.0
34,jaguar,sedan,113.0,199.6,dohc,six,176.0,15.0,35550.0
37,mazda,hatchback,93.1,159.1,ohc,four,68.0,31.0,6095.0
38,mazda,hatchback,93.1,159.1,ohc,four,68.0,31.0,6795.0


Har doimgidek, `axis=1` parametri yordamida qatorlarni emas, ustunlarni tashlab yuborishimiz mumkin:

In [11]:
df.dropna(axis=1)

,company,body-style,num-of-cylinders
index,,,
0,alfa-romero,convertible,four
1,alfa-romero,convertible,four
2,alfa-romero,hatchback,six
3,audi,sedan,four
4,audi,sedan,five
...,...,...,...
81,volkswagen,sedan,four
82,volkswagen,sedan,four
86,volkswagen,sedan,four


Yuqoridagi natijadan ko'rishimiz mumkinki, `NaN` bor barcha ustunlar tushib qoldi va DF deyarli yaroqsiz holatga keldi. 

`thresh` parametri yordamida biz `NaN` bo'lmagan qiymatlarining soniga chegara belgilashimiz mumkin. Misol uchun, quyidagi kodimiz `NaN` bo'lmagan qiymatlar 48 dan kam bo'lgan ustunlarni tashlab yuboradi. 

In [12]:
df.dropna(axis=1, thresh=50)

,company,body-style,num-of-cylinders,price
index,,,,
0,alfa-romero,convertible,four,13495.0
1,alfa-romero,convertible,four,16500.0
2,alfa-romero,hatchback,six,16500.0
3,audi,sedan,four,13950.0
4,audi,sedan,five,17450.0
...,...,...,...,...
81,volkswagen,sedan,four,7975.0
82,volkswagen,sedan,four,7995.0
86,volkswagen,sedan,four,9995.0


`subset` parametri yordamida aynan qaysi ustunlarni tekshirishni ham ko'rsatish mumkin. Quyidagi misolda faqatgina `length` va `body-style` ustunlarida `NaN` qiymati bor qatorlarni tashlab yuboramiz.

In [16]:
df.dropna(subset=['length', 'body-style'])

,company,body-style,wheel-base,length,engine-type,num-of-cylinders,horsepower,average-mileage,price
index,,,,,,,,,
0,alfa-romero,convertible,88.6,168.8,dohc,four,NaN,21.0,13495.0
1,alfa-romero,convertible,NaN,168.8,dohc,four,111.0,21.0,16500.0
2,alfa-romero,hatchback,94.5,171.2,ohcv,six,154.0,NaN,16500.0
3,audi,sedan,99.8,176.6,ohc,four,102.0,24.0,13950.0
4,audi,sedan,99.4,176.6,ohc,five,NaN,NaN,17450.0
5,audi,sedan,99.8,177.3,ohc,five,110.0,19.0,15250.0
6,audi,wagon,105.8,192.7,ohc,five,110.0,19.0,18920.0
10,bmw,sedan,NaN,176.8,ohc,four,101.0,23.0,16925.0
11,bmw,sedan,101.2,176.8,ohc,six,121.0,NaN,20970.0


Yuqoridagi misollarda bizning asl `df` jadvalimiz o'zgarishsiz qolmoqda, o'zgarishlarni `df` da saqlash uchun `inplace=True` parametridan foydalanamiz.

In [18]:
df.dropna(subset=['price'], inplace=True)

In [19]:
df.shape

(58, 9)

In [20]:
df.head()

,company,body-style,wheel-base,length,engine-type,num-of-cylinders,horsepower,average-mileage,price
index,,,,,,,,,
0,alfa-romero,convertible,88.6,168.8,dohc,four,NaN,21.0,13495.0
1,alfa-romero,convertible,NaN,168.8,dohc,four,111.0,21.0,16500.0
2,alfa-romero,hatchback,94.5,171.2,ohcv,six,154.0,NaN,16500.0
3,audi,sedan,99.8,176.6,ohc,four,102.0,24.0,13950.0
4,audi,sedan,99.4,176.6,ohc,five,NaN,NaN,17450.0
